In [33]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [5]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    df = pd.read_csv('university_towns.txt', header=None, sep='\t')
    df.columns = ['combined']
    df['combined'] = df.apply(lambda x: x['combined']+'\t' if '[edit]' in x['combined'] else '\t'+x['combined'],
                              axis=1)
    df['State'] = df['combined'].str.split("\t").str[0] \
                                .str.split('[').str[0]
    df['RegionName'] = df['combined'].str.split("\t").str[1] \
                                     .str.split(' \(').str[0]
    df = df.applymap(lambda x: np.nan if (x.isspace() or x=='') else x)
    df['State'] = df['State'].fillna(method='ffill')
    df = df[df['RegionName'].notnull()].drop(['combined'], axis=1)
   
    
    return df

df = get_list_of_university_towns()



In [59]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel('gdplev.xls', skiprows=7)
    df = df.iloc[:,4:7]
    df.columns = ['Quarter', 'GDP in billions of current dollars', 'GDP in billions of chained 2009 dollars']
    df = df.set_index('Quarter')
    df = df[df.index > '2000']
    recession = []
    for i in range (np.size(df['GDP in billions of current dollars']) - 2):
        if ((df['GDP in billions of current dollars'].iloc[i+1] - df['GDP in billions of current dollars'].iloc[i]) < 0 and
            (df['GDP in billions of current dollars'].iloc[i+2] - df['GDP in billions of current dollars'].iloc[i+1]) < 0):
            recession += [df.iloc[i+1].name]
    return recession[0]
get_recession_start()

'2008q4'

In [61]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel('gdplev.xls', skiprows=7)
    df = df.iloc[:,4:7]
    df.columns = ['Quarter', 'GDP in billions of current dollars', 'GDP in billions of chained 2009 dollars']
    df = df.set_index('Quarter')
    df = df[df.index >= get_recession_start()]
    non_recession = []
    for i in range (np.size(df['GDP in billions of current dollars']) - 2):
        if ((df['GDP in billions of current dollars'].iloc[i+1] - df['GDP in billions of current dollars'].iloc[i]) > 0 and
            (df['GDP in billions of current dollars'].iloc[i+2] - df['GDP in billions of current dollars'].iloc[i+1]) > 0):
            non_recession += [df.iloc[i+1].name]
    return non_recession[0]   


get_recession_end()

'2009q3'

In [10]:
for i in range (np.size(df['GDP in billions of current dollars']) - 2):
    if ((df['GDP in billions of current dollars'].iloc[i+1] - df['GDP in billions of current dollars'].iloc[i]) < 0 and
       (df['GDP in billions of current dollars'].iloc[i+2] - df['GDP in billions of current dollars'].iloc[i+1]) < 0):
        df['Recession'].iloc[i + 1] = 'R'

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [40]:
goulet = pd.Series()
goulet.append()


Series([], dtype: float64)

In [82]:
df['GDP in billions of current dollars'].iloc[1]

10278.3

In [16]:
x

'Alabama[edit]'

In [17]:
x.endswith('[edit]')

True